In [1]:
import pandas as pd
import geopandas as gpd

# Reading in the data
starbucks = pd.read_csv('/home/jake_ockerby/Python/starbucks/starbucks_locations.csv')
starbucks.head()

/home/jake_ockerby/.local/lib/python3.5/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [2]:
# Finding missing values
print(starbucks.isnull().sum())

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


In [3]:
rows_missing = starbucks[starbucks['Latitude'].isnull()]
rows_missing

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [4]:
from geopandas.tools import geocode
from geopy.geocoders import Nominatim

# Creating a geocoder that takes an address and coverts into latitude and longitude
def my_geocoder(row):
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(row)
    return pd.Series({'Longitude': location.longitude, 'Latitude': location.latitude})

# Using the created function to fill in the missing values of berkeley
berkeley_locations = rows_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)
starbucks[starbucks['City']=='Berkeley']

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,-122.268230,37.868839
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,-122.280009,37.891471
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,-122.269869,37.881177
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,-122.259370,37.855947
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,-122.266079,37.870410


In [5]:
import folium 
from folium import Marker
from folium.plugins import MarkerCluster

# Creating a function to visualise maps from a source of your choice
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [6]:
berkeley_map = folium.Map(location=[37.88,-122.26], zoom_start=13)

# For each address in berkeley add a marker to the map
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(berkeley_map)
    
embed_map(berkeley_map, 'www.openstreetmap.html')

In [7]:
# Reading in the GeoDataFrame of california counties

CA_counties = gpd.read_file('/home/jake_ockerby/Python/starbucks/CA_counties/CA_counties.shp', index_col='GEOID')
CA_counties['GEOID'] = CA_counties['GEOID'].astype('int64')
CA_counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,107,00277318,6107,Tulare,Tulare County,06,H1,G4020,None,47300,None,A,12494707314,37391604,+36.2288317,-118.7810618,"POLYGON ((-118.36059 36.74477, -118.36013 36.7..."
1,06,009,01675885,6009,Calaveras,Calaveras County,06,H1,G4020,None,None,None,A,2641820029,43810423,+38.1846184,-120.5593996,"POLYGON ((-120.01995 38.43352, -120.02020 38.4..."
2,06,047,00277288,6047,Merced,Merced County,06,H1,G4020,None,32900,None,A,5011554680,112760479,+37.1948063,-120.7228019,"POLYGON ((-120.05205 37.18311, -120.05215 37.1..."
3,06,079,00277304,6079,San Luis Obispo,San Luis Obispo County,06,H1,G4020,None,42020,None,A,8543230300,820974619,+35.3852268,-120.4475409,"POLYGON ((-120.21398 35.78928, -120.20627 35.7..."
4,06,097,01657246,6097,Sonoma,Sonoma County,06,H1,G4020,488,42220,None,A,4081430061,497530414,+38.5250258,-122.9376050,"MULTIPOLYGON (((-122.51299 38.13180, -122.4956..."
5,06,041,00277285,6041,Marin,Marin County,06,H1,G4020,488,41860,41884,A,1347585499,797420416,+38.0518169,-122.7459738,"MULTIPOLYGON (((-123.02325 38.29560, -123.0488..."
6,06,023,01681908,6023,Humboldt,Humboldt County,06,H1,G4020,None,21700,None,A,9241033283,1254256391,+40.7066731,-123.9258181,"POLYGON ((-123.66136 41.38209, -123.66089 41.3..."
7,06,051,00277290,6051,Mono,Mono County,06,H1,G4020,None,None,None,A,7896837613,214695460,+37.9158363,-118.8751668,"POLYGON ((-119.32840 38.53479, -119.30878 38.5..."
8,06,015,01682074,6015,Del Norte,Del Norte County,06,H1,G4020,None,18860,None,A,2606493818,578525693,+41.7499033,-123.9809983,"POLYGON ((-123.51791 42.00085, -123.51793 42.0..."
9,06,011,01675902,6011,Colusa,Colusa County,06,H1,G4020,None,None,None,A,2980379346,14581041,+39.1777385,-122.2375629,"POLYGON ((-121.89001 39.38386, -121.88995 39.3..."


In [8]:
# Reading california demographic data and merging to the CA_counties data into one DataFrame

CA_pop = pd.read_csv('/home/jake_ockerby/Python/starbucks/CA_county_population.csv', index_col='GEOID')
CA_high_earners = pd.read_csv('/home/jake_ockerby/Python/starbucks/CA_county_high_earners.csv', index_col='GEOID')
CA_median_age = pd.read_csv('/home/jake_ockerby/Python/starbucks/CA_county_median_age.csv', index_col='GEOID')

In [9]:
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")
CA_stats.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,population,high_earners,median_age
0,06,107,00277318,6107,Tulare,Tulare County,06,H1,G4020,None,...,None,A,12494707314,37391604,+36.2288317,-118.7810618,"POLYGON ((-118.36059 36.74477, -118.36013 36.7...",465861,9056,30.6
1,06,009,01675885,6009,Calaveras,Calaveras County,06,H1,G4020,None,...,None,A,2641820029,43810423,+38.1846184,-120.5593996,"POLYGON ((-120.01995 38.43352, -120.02020 38.4...",45602,2046,51.6
2,06,047,00277288,6047,Merced,Merced County,06,H1,G4020,None,...,None,A,5011554680,112760479,+37.1948063,-120.7228019,"POLYGON ((-120.05205 37.18311, -120.05215 37.1...",274765,5933,30.8
3,06,079,00277304,6079,San Luis Obispo,San Luis Obispo County,06,H1,G4020,None,...,None,A,8543230300,820974619,+35.3852268,-120.4475409,"POLYGON ((-120.21398 35.78928, -120.20627 35.7...",284010,15110,39.0
4,06,097,01657246,6097,Sonoma,Sonoma County,06,H1,G4020,488,...,None,A,4081430061,497530414,+38.5250258,-122.9376050,"MULTIPOLYGON (((-122.51299 38.13180, -122.4956...",499942,32713,41.4


In [10]:
# Making a new column called density

CA_stats["density"] = CA_stats["population"] / (CA_stats['ALAND']/1000000)

In [11]:
from pyproj import CRS

# Selecting counties based on high income, low median age and high density
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]
sel_counties.crs = CRS('epsg:4326')
sel_counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,...,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,population,high_earners,median_age,density
18,06,075,00277302,6075,San Francisco,San Francisco County,06,H6,G4020,488,...,C,121399974,479190304,+37.7272391,-123.0322294,"MULTIPOLYGON (((-122.34647 37.72522, -122.3612...",883305,114989,38.3,7275.990026
38,06,073,00277301,6073,San Diego,San Diego County,06,H1,G4020,None,...,A,10895054821,826347910,+33.0236041,-116.7761174,"POLYGON ((-116.08517 33.42593, -116.08516 33.4...",3343364,194676,35.4,306.869865
55,06,037,00277283,6037,Los Angeles,Los Angeles County,06,H1,G4020,348,...,A,10509870524,1795012008,+34.1963983,-118.2618616,"MULTIPOLYGON (((-118.35096 32.81920, -118.4201...",10105518,501413,36.0,961.526403
56,06,059,00277294,6059,Orange,Orange County,06,H1,G4020,348,...,A,2047561073,407916803,+33.6756872,-117.7772068,"POLYGON ((-117.78329 33.94641, -117.78307 33.9...",3185968,233459,37.5,1555.981915


In [12]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = CRS('epsg:4326')

# All starbucks stores in the selceted counties
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
locations_of_interest

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry,index_right,STATEFP,COUNTYFP,...,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,population,high_earners,median_age,density
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
14,76365-97162,Target Alhambra T-184,1220 West Main Street Alhambra CA,Alhambra,-118.14,34.09,POINT (-118.14000 34.09000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
15,6794-41839,Fremont Ave & Mission Rd,"1131 S Fremont Ave, A Alhambra CA",Alhambra,-118.15,34.08,POINT (-118.15000 34.08000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
16,11220-104633,"Atlantic & Valley, Alhambra",1410 South Atlantic Blvd. Alhambra CA,Alhambra,-118.13,34.08,POINT (-118.13000 34.08000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
17,47671-258708,Albertsons - Alhambras 6543,2400 W. Commonwealth Alhambra CA,Alhambra,-118.15,34.09,POINT (-118.15000 34.09000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
18,9385-96258,"Valley & Almansor, Alhambra",810 E VALLEY BLVD ALHAMBRA CA,ALHAMBRA,-118.11,34.08,POINT (-118.11000 34.08000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
19,5380-673,Main & 1st - Alhambra,101 W. Main Street Alhambra CA,Alhambra,-118.13,34.09,POINT (-118.13000 34.09000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
67,73672-96295,Albertsons-Arcadia #6561,298 E Live Oak Ave Arcadia CA,Arcadia,-118.02,34.11,POINT (-118.02000 34.11000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403
68,5497-1534,300 E. Huntington Dr. - Arcadia,"300 E Huntington Drive, B15, Santa Anita Arcad...",Arcadia,-118.02,34.14,POINT (-118.02000 34.14000),55,06,037,...,31084,A,10509870524,1795012008,+34.1963983,-118.2618616,10105518,501413,36.0,961.526403


In [13]:
len(locations_of_interest)

1303

In [14]:
# expected number of stores in each county using population data

sf_expected = (883305/sel_counties['population'].sum())*1303
sd_expected = (3343364/sel_counties['population'].sum())*1303
la_expected = (10105518/sel_counties['population'].sum())*1303
or_expected = (3185968/sel_counties['population'].sum())*1303

print('San Francisco: ', sf_expected)
print('San Diego: ', sd_expected)
print('Los Angeles: ', la_expected)
print('Orange County: ', or_expected)

San Francisco:  65.70020729922757
San Diego:  248.67934391492713
Los Angeles:  751.6482160364491
Orange County:  236.97223274939626


In [15]:
import math

CA_map = folium.Map(location=[37,-120], zoom_start=6)
mc = MarkerCluster()

for idx, row in locations_of_interest.iterrows():
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
        
CA_map.add_child(mc)
embed_map(CA_map, 'www.openstreetmap.html')

#The map and the previous data suggests that Orange County would be the best candidate for more starbucks stores.
#However, Orange County has a very high density so the benefits another store provides may be minimal and therefore
#not be cost effective.